# Aula 11.Ex02 - Malhas e Texturas - Mapeamento de Texturas

### Primeiro, vamos importar as bibliotecas necessárias.
Verifique no código anterior um script para instalar as dependências necessárias (OpenGL e GLFW) antes de prosseguir.

In [ ]:
import glfw
from OpenGL.GL import *
import OpenGL.GL.shaders
import numpy as np
import glm
import math
from PIL import Image

### Inicializando janela

In [ ]:
glfw.init()
glfw.window_hint(glfw.VISIBLE, glfw.FALSE);
altura = 1600
largura = 1200
window = glfw.create_window(largura, altura, "SCC0250 - Trabalho 2", None, None)
glfw.make_context_current(window)

### GLSL (OpenGL Shading Language)

Aqui veremos nosso primeiro código GLSL.

É uma linguagem de shading de alto nível baseada na linguagem de programação C.

Nós estamos escrevendo código GLSL como se "strings" de uma variável (mas podemos ler de arquivos texto). Esse código, depois, terá que ser compilado e linkado ao nosso programa. 

Iremos aprender GLSL conforme a necessidade do curso. Usarmos uma versão do GLSL mais antiga, compatível com muitos dispositivos.

### GLSL para Vertex Shader

No Pipeline programável, podemos interagir com Vertex Shaders.

No código abaixo, estamos fazendo o seguinte:

* Definindo uma variável chamada position do tipo vec3.
* Definindo matrizes Model, View e Projection que acumulam transformações geométricas 3D e permitem navegação no cenário.
* void main() é o ponto de entrada do nosso programa (função principal)
* gl_Position é uma variável especial do GLSL. Variáveis que começam com 'gl_' são desse tipo. Nesse caso, determina a posição de um vértice. Observe que todo vértice tem 4 coordenadas, por isso nós combinamos nossa variável vec2 com uma variável vec4. Além disso, nós modificamos nosso vetor com base nas transformações Model, View e Projection.

In [ ]:
vertex_code = """
        attribute vec3 position;
        attribute vec2 texture_coord;
        varying vec2 out_texture;
                
        uniform mat4 model;
        uniform mat4 view;
        uniform mat4 projection;        
        
        void main(){
            gl_Position = projection * view * model * vec4(position,1.0);
            out_texture = vec2(texture_coord);
        }
        """

### GLSL para Fragment Shader

No Pipeline programável, podemos interagir com Fragment Shaders.

No código abaixo, estamos fazendo o seguinte:

* void main() é o ponto de entrada do nosso programa (função principal)
* gl_FragColor é uma variável especial do GLSL. Variáveis que começam com 'gl_' são desse tipo. Nesse caso, determina a cor de um fragmento. Nesse caso é um ponto, mas poderia ser outro objeto (ponto, linha, triangulos, etc).

### Possibilitando modificar a cor.

Nos exemplos anteriores, a variável gl_FragColor estava definida de forma fixa (com cor R=0, G=0, B=0).

Agora, nós vamos criar uma variável do tipo "uniform", de quatro posições (vec4), para receber o dado de cor do nosso programa rodando em CPU.

In [ ]:
fragment_code = """
        uniform vec4 color;
        varying vec2 out_texture;
        uniform sampler2D samplerTexture;
        
        void main(){
            vec4 texture = texture2D(samplerTexture, out_texture);
            gl_FragColor = texture;
        }
        """

### Requisitando slot para a GPU para nossos programas Vertex e Fragment Shaders

In [ ]:
# Request a program and shader slots from GPU
program  = glCreateProgram()
vertex   = glCreateShader(GL_VERTEX_SHADER)
fragment = glCreateShader(GL_FRAGMENT_SHADER)


### Associando nosso código-fonte aos slots solicitados

In [ ]:
# Set shaders source
glShaderSource(vertex, vertex_code)
glShaderSource(fragment, fragment_code)

### Compilando o Vertex Shader

Se há algum erro em nosso programa Vertex Shader, nosso app para por aqui.

In [ ]:
# Compile shaders
glCompileShader(vertex)
if not glGetShaderiv(vertex, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(vertex).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Vertex Shader")


### Compilando o Fragment Shader

Se há algum erro em nosso programa Fragment Shader, nosso app para por aqui.

In [ ]:
glCompileShader(fragment)
if not glGetShaderiv(fragment, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(fragment).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Fragment Shader")

### Associando os programas compilado ao programa principal

In [ ]:
# Attach shader objects to the program
glAttachShader(program, vertex)
glAttachShader(program, fragment)


### Linkagem do programa

In [ ]:
# Build program
glLinkProgram(program)
if not glGetProgramiv(program, GL_LINK_STATUS):
    print(glGetProgramInfoLog(program))
    raise RuntimeError('Linking error')
    
# Make program the default program
glUseProgram(program)

### Preparando dados para enviar a GPU

Nesse momento, nós compilamos nossos Vertex e Program Shaders para que a GPU possa processá-los.

Por outro lado, as informações de vértices geralmente estão na CPU e devem ser transmitidas para a GPU.


### Carregando Modelos (vértices e texturas) a partir de Arquivos

A função abaixo carrega modelos a partir de arquivos no formato WaveFront.


Para saber mais sobre o modelo, acesse: https://en.wikipedia.org/wiki/Wavefront_.obj_file


Nos slides e vídeo-aula da Aula 11 - Parte 1, nós descrevemos o funcionamento desse formato.

In [ ]:
def load_model_from_file(filename):
    """Loads a Wavefront OBJ file. """
    objects = {}
    vertices = []
    texture_coords = []
    faces = []

    material = None

    # abre o arquivo obj para leitura
    for line in open(filename, "r"): ## para cada linha do arquivo .obj
        if line.startswith('#'): continue ## ignora comentarios
        values = line.split() # quebra a linha por espaço
        if not values: continue


        ### recuperando vertices
        if values[0] == 'v':
            vertices.append(values[1:4])


        ### recuperando coordenadas de textura
        elif values[0] == 'vt':
            texture_coords.append(values[1:3])

        ### recuperando faces 
        elif values[0] in ('usemtl', 'usemat'):
            material = values[1]
        elif values[0] == 'f':
            face = []
            face_texture = []
            for v in values[1:]:
                w = v.split('/')
                face.append(int(w[0]))
                if len(w) >= 2 and len(w[1]) > 0:
                    face_texture.append(int(w[1]))
                else:
                    face_texture.append(0)

            faces.append((face, face_texture, material))

    model = {}
    model['vertices'] = vertices
    model['texture'] = texture_coords
    model['faces'] = faces

    return model


In [ ]:
glEnable(GL_TEXTURE_2D)
qtd_texturas = 10
textures = glGenTextures(qtd_texturas)

def load_texture_from_file(texture_id, img_textura):
    glBindTexture(GL_TEXTURE_2D, texture_id)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_S, GL_REPEAT)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_T, GL_REPEAT)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MIN_FILTER, GL_LINEAR)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MAG_FILTER, GL_LINEAR)
    img = Image.open(img_textura)
    img_width = img.size[0]
    img_height = img.size[1]
    image_data = img.tobytes("raw", "RGB", 0, -1)
    #image_data = np.array(list(img.getdata()), np.uint8)
    glTexImage2D(GL_TEXTURE_2D, 0, GL_RGB, img_width, img_height, 0, GL_RGB, GL_UNSIGNED_BYTE, image_data)

    return
    

### A lista abaixo armazena todos os vertices carregados dos arquivos

In [ ]:
id_textura = 0
vertices_list = []    
textures_coord_list = []

### Vamos carregar cada modelo e definir funções para desenhá-los

In [ ]:
# Insere modelo contido no arquivo "file_name" na lista de vértices
def insert_model(file_name,texture_name):
    global id_textura, vertices_list, textures_coord_list

    # atribuindo o id do objeto de acordo com o valor atual
    object_id = id_textura
    id_textura += 1

    modelo = load_model_from_file(file_name)

    vi = len(vertices_list)   

    ### inserindo vértices do modelo no vetor de vértices
    print('Processando modelo ' + str(file_name) + ' ...')
    print('Vértice inicial: ' + str(len(vertices_list)))

    for face in modelo['faces']:
        for vertice_id in face[0]:
            vertices_list.append( modelo['vertices'][vertice_id-1] )
        for texture_id in face[1]:
            textures_coord_list.append( modelo['texture'][texture_id-1] )    

    print('Vértice final: ' + str(len(vertices_list)))

    vf = len(vertices_list)        

    load_texture_from_file(object_id,texture_name)

    return vi,(vf-vi),object_id
    

In [ ]:
# Insere modelo contido com múltiplas texturas no arquivo "file_name" na lista de vértices
def insert_model_mult_texture(file_name,texture_names):
    global id_textura, vertices_list, textures_coord_list

    n = len(texture_names)      # número de arquivos de textura
    object_id = []              # lista de ids atribuídos às partes do objeto
    vi = []                     # lista de vértices iniciais dos componentes do objeto
    qtd = []                    # lista da quantidade de vértices de cada componente do objeto
    faces_visited = []          # lista de faces visitadas
    component = 0               # número de componentes

     # atribuindo o id do objeto de acordo com o valor atual e o número de arquivos de textura
    for i in range(0,n):
        object_id.append(id_textura)
        id_textura += 1

    modelo = load_model_from_file(file_name) 

    ### inserindo vértices do modelo no vetor de vértices
    print('Processando modelo ' + str(file_name) + ' ...')

    for face in modelo['faces']:
        if face[2] not in faces_visited:
            print('Componente encontrado no vértice ' + str(len(vertices_list)))
            vi.append(len(vertices_list))
            faces_visited.append(face[2])
            component += 1
        if component > 1:
            qtd.append(len(vertices_list) - vi[component - 2])
        for vertice_id in face[0]:
            vertices_list.append( modelo['vertices'][vertice_id-1] )
        for texture_id in face[1]:
            textures_coord_list.append( modelo['texture'][texture_id-1] )    

    print('Vértice final: ' + str(len(vertices_list)))

    qtd.append(len(vertices_list) - vi[component - 1])      

    for i in range(0,n):
        load_texture_from_file(object_id[i],texture_names[i])

    return vi,qtd,object_id
    

In [ ]:
# Função para retornar lista com três elementos
def triple(first,second,third):

    lista = [None] * 3
    lista[0] = first
    lista[1] = second
    lista[2] = third

    return lista

In [ ]:

#-------------CURRY POT-------------#

# lista com os arquivos de textura
#textures_curry = []
#textures_curry.append('modelos/curry/curry-2.png')
#textures_curry.append('modelos/curry/curry-1.png')
#textures_curry.append('modelos/curry/curry-3.png')
#angle_curry = 0.0                                                                                   # ângulo de rotação
#r_curry = triple(0.0, 0.0, 1.0)                                                                     # rotação
#t_curry = triple(0.0, -1.0, 0.0)                                                                    # translação
#s_curry = triple(0.01,0.01,0.01)                                                                    # escala
#vi_curry, qtd_curry, id_curry = insert_model_mult_texture('modelos/curry/pot.obj',textures_curry)   # vértices


In [ ]:

#-------------MUSEUM-------------#

angle_museum = 0.0                                                                                     # ângulo de rotação
r_museum = triple(0.0, 0.0, 1.0)                                                                        # rotação
t_museum = triple(0.0, -1.0, 0.0)                                                                       # translação
s_museum = triple(0.1,0.1,0.1)                                                                          # escala
vi_museum, qtd_museum, id_museum = insert_model('modelos/museum/museum.obj','modelos/museum/museum.jpeg')   # vértices

In [ ]:

#-------------DUCK-------------#

angle_duck = 0.0                                                                             # ângulo de rotação
r_duck = triple(0.0, 0.0, 1.0)                                                               # rotação
t_duck = triple(0.1, 0.7, 0.0)                                                               # translação
s_duck = triple(0.28,0.28,0.28)                                                              # escala
vi_duck, qtd_duck, id_duck = insert_model('modelos/duck/duck.obj','modelos/duck/duck.png')   # vértices

In [ ]:

#-------------SEEDS-------------#

angle_seeds = 0.0                                                                                 # ângulo de rotação
r_seeds = triple(0.0,0.0, 1.0)                                                                    # rotação
t_seeds = triple(4.0,-1.0,2.0)                                                                    # translação
s_seeds = triple(12.0,12.0,12.0)                                                                  # escala
vi_seeds, qtd_seeds, id_seeds = insert_model('modelos/seeds/seeds.obj','modelos/seeds/seeds.png') # vértices

In [ ]:

#-------------APPLES-------------#

angle_apple = triple(10.0,0.0,15.0)                                                               # ângulo de rotação
r_apple = triple([1.0,0.0,1.0],[0.0,0.0,1.0],[0.0,0.0,1.0])                                       # rotação                                                                    
t_apple = triple([3.6,-1.4,0.2],[4.2,-1.0,-0.2],[3.6,-2.0,-0.5])                                  # translação
s_apple = triple([4.0,4.0,4.0],[4.0,4.0,4.0],[4.0,4.0,4.0])                                       # escala
vi_apple, qtd_apple, id_apple = insert_model('modelos/apple/apple.obj','modelos/apple/apple.png') # vértices

In [ ]:

#-------------CAMPFIRE-------------#

angle_campfire = 0.0                                                                                                   # ângulo de rotação
r_campfire = triple(0.0,0.0,1.0)                                                                                       # rotação
t_campfire = triple(8.0,-1.0,-0.2)                                                                                     # translação
s_campfire = triple(0.02,0.02,0.02)                                                                                    # escala
vi_campfire, qtd_campfire, id_campfire = insert_model('modelos/campfire/campfire.obj','modelos/campfire/campfire.png') # vértices

In [ ]:
# House:
# Author: PoossyJooce
# License: https://creativecommons.org/licenses/by/4.0/

# Duck:
# Author:tantangula
# License: https://creativecommons.org/licenses/by/4.0/

### Para enviar nossos dados da CPU para a GPU, precisamos requisitar slots.

Nós agora vamos requisitar dois slots.
* Um para enviar coordenadas dos vértices.
* Outros para enviar coordenadas de texturas.

In [ ]:
# Request a buffer slot from GPU
buffer = glGenBuffers(2)


###  Enviando coordenadas de vértices para a GPU

In [ ]:
vertices = np.zeros(len(vertices_list), [("position", np.float32, 3)])
vertices['position'] = vertices_list

# Upload data
glBindBuffer(GL_ARRAY_BUFFER, buffer[0])
glBufferData(GL_ARRAY_BUFFER, vertices.nbytes, vertices, GL_STATIC_DRAW)
stride = vertices.strides[0]
offset = ctypes.c_void_p(0)
loc_vertices = glGetAttribLocation(program, "position")
glEnableVertexAttribArray(loc_vertices)
glVertexAttribPointer(loc_vertices, 3, GL_FLOAT, False, stride, offset)


###  Enviando coordenadas de textura para a GPU

In [ ]:
textures = np.zeros(len(textures_coord_list), [("position", np.float32, 2)]) # duas coordenadas
textures['position'] = textures_coord_list

# Upload data
glBindBuffer(GL_ARRAY_BUFFER, buffer[1])
glBufferData(GL_ARRAY_BUFFER, textures.nbytes, textures, GL_STATIC_DRAW)
stride = textures.strides[0]
offset = ctypes.c_void_p(0)
loc_texture_coord = glGetAttribLocation(program, "texture_coord")
glEnableVertexAttribArray(loc_texture_coord)
glVertexAttribPointer(loc_texture_coord, 2, GL_FLOAT, False, stride, offset)


### Desenhando nossos modelos
* Cada modelo tem um Model para posicioná-los no mundo.
* É necessário saber qual a posição inicial e total de vértices de cada modelo
* É necessário indicar qual o ID da textura do modelo


In [ ]:
# Desenha o objeto com as propriedades especificadas
def draw_object(angle,r,s,t,vi,qtd,texture_id):

    global vertices

    # aplica a matriz model
    mat_model = model(angle, r, t, s)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model) 

    # define o id da textura do modelo
    glBindTexture(GL_TEXTURE_2D, texture_id)

    # desenha o objeto 
    glDrawArrays(GL_TRIANGLES, vi, qtd)

    return  

In [ ]:
# Desenha objeto com múltiplas texturas e com as propriedades especificadas
def draw_object_mult_texture(angle,r,s,t,vi,qtd,texture_id):

    global vertices

    # aplica a matriz model
    mat_model = model(angle, r, t, s)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model) 

    n = len(texture_id)

    for i in range(0,n):
        glBindTexture(GL_TEXTURE_2D, texture_id[i]) # define o id da textura do modelo
        glDrawArrays(GL_TRIANGLES, vi[i], qtd[i])   # desenha o objeto 

    return  

### Eventos para modificar a posição da câmera
* Usei as teclas A, S, D e W para movimentação no espaço tridimensional
* Usei a posição do mouse para "direcionar" a câmera

In [ ]:
cameraPos   = glm.vec3(0.0,  0.0,  1.0);
cameraFront = glm.vec3(0.0,  0.0, -1.0);
cameraUp    = glm.vec3(0.0,  1.0,  0.0);


polygonal_mode = False

def key_event(window,key,scancode,action,mods):
    global cameraPos, cameraFront, cameraUp, polygonal_mode
    
    cameraSpeed = 0.2
    if key == 87 and (action==1 or action==2): # tecla W
        cameraPos += cameraSpeed * cameraFront
    
    if key == 83 and (action==1 or action==2): # tecla S
        cameraPos -= cameraSpeed * cameraFront
    
    if key == 65 and (action==1 or action==2): # tecla A
        cameraPos -= glm.normalize(glm.cross(cameraFront, cameraUp)) * cameraSpeed
        
    if key == 68 and (action==1 or action==2): # tecla D
        cameraPos += glm.normalize(glm.cross(cameraFront, cameraUp)) * cameraSpeed
        
    if key == 80 and action==1 and polygonal_mode==True:
        polygonal_mode=False
    else:
        if key == 80 and action==1 and polygonal_mode==False:
            polygonal_mode=True  
        
firstMouse = True
yaw = -90.0 
pitch = 0.0
lastX =  largura/2
lastY =  altura/2

def mouse_event(window, xpos, ypos):
    global firstMouse, cameraFront, yaw, pitch, lastX, lastY
    if firstMouse:
        lastX = xpos
        lastY = ypos
        firstMouse = False

    xoffset = xpos - lastX
    yoffset = lastY - ypos
    lastX = xpos
    lastY = ypos

    sensitivity = 0.3 
    xoffset *= sensitivity
    yoffset *= sensitivity

    yaw += xoffset;
    pitch += yoffset;

    
    if pitch >= 90.0: pitch = 90.0
    if pitch <= -90.0: pitch = -90.0

    front = glm.vec3()
    front.x = math.cos(glm.radians(yaw)) * math.cos(glm.radians(pitch))
    front.y = math.sin(glm.radians(pitch))
    front.z = math.sin(glm.radians(yaw)) * math.cos(glm.radians(pitch))
    cameraFront = glm.normalize(front)


    
glfw.set_key_callback(window,key_event)
glfw.set_cursor_pos_callback(window, mouse_event)


### Matrizes Model, View e Projection

Teremos uma aula específica para entender o seu funcionamento.

In [ ]:
def model(angle, r, t, s):
    
    angle = math.radians(angle)
    
    matrix_transform = glm.mat4(1.0)                                                   # instanciando uma matriz identidade
    matrix_transform = glm.rotate(matrix_transform, angle, glm.vec3(r[0], r[1], r[2])) # aplicando rotacao
    matrix_transform = glm.translate(matrix_transform, glm.vec3(t[0], t[1], t[2]))     # aplicando translacao     
    matrix_transform = glm.scale(matrix_transform, glm.vec3(s[0], s[1], s[2]))         # aplicando escala
    matrix_transform = np.array(matrix_transform).T                                    # pegando a transposta da matriz (glm trabalha com ela invertida)
    
    return matrix_transform

def view():
    global cameraPos, cameraFront, cameraUp
    mat_view = glm.lookAt(cameraPos, cameraPos + cameraFront, cameraUp);
    mat_view = np.array(mat_view)
    return mat_view

def projection():
    global altura, largura
    # perspective parameters: fovy, aspect, near, far
    mat_projection = glm.perspective(glm.radians(45.0), largura/altura, 0.1, 1000.0)
    mat_projection = np.array(mat_projection)    
    return mat_projection
    

### Nesse momento, nós exibimos a janela!


In [ ]:
glfw.show_window(window)
glfw.set_cursor_pos(window, lastX, lastY)

### Loop principal da janela.
Enquanto a janela não for fechada, esse laço será executado. É neste espaço que trabalhamos com algumas interações com a OpenGL.

In [ ]:
glEnable(GL_DEPTH_TEST) ### importante para 3D
   
while not glfw.window_should_close(window):

    glfw.poll_events() 
    
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)
    
    glClearColor(1.0, 1.0, 1.0, 1.0)
    
    if polygonal_mode == True: glPolygonMode(GL_FRONT_AND_BACK,GL_LINE)
    if polygonal_mode == False: glPolygonMode(GL_FRONT_AND_BACK,GL_FILL)
    
    #museum
    draw_object(angle_museum,r_museum,s_museum,t_museum,vi_museum,qtd_museum,id_museum)
    #duck
    draw_object(angle_duck,r_duck,s_duck,t_duck,vi_duck,qtd_duck,id_duck)
    #seeds
    draw_object(angle_seeds,r_seeds,s_seeds,t_seeds,vi_seeds,qtd_seeds,id_seeds)
    #apples
    draw_object(angle_apple[0],r_apple[0],s_apple[0],t_apple[0],vi_apple,qtd_apple,id_apple)
    draw_object(angle_apple[1],r_apple[1],s_apple[1],t_apple[1],vi_apple,qtd_apple,id_apple)
    draw_object(angle_apple[2],r_apple[2],s_apple[2],t_apple[2],vi_apple,qtd_apple,id_apple)
    #campfire
    draw_object(angle_campfire,r_campfire,s_campfire,t_campfire,vi_campfire,qtd_campfire,id_campfire)

    mat_view = view()
    loc_view = glGetUniformLocation(program, "view")
    glUniformMatrix4fv(loc_view, 1, GL_FALSE, mat_view)

    mat_projection = projection()
    loc_projection = glGetUniformLocation(program, "projection")
    glUniformMatrix4fv(loc_projection, 1, GL_FALSE, mat_projection)    
    
    glfw.swap_buffers(window)

glfw.terminate()